# Neural network training on csv data
After running the first cell, you will have to sign in with google and copy a code

In [1]:
!pip install -Uqq fastbook
import fastbook
from fastbook import *
fastbook.setup_book()
from fastai.tabular.all import *

     |████████████████████████████████| 727kB 17.7MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 1.2MB 21.2MB/s 
     |████████████████████████████████| 194kB 57.0MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 12.8MB 235kB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive


Replace the `path` variable with a relative path, or one relative to `gdrive` to access data from Google Drive

In [3]:
# following tutorial from: https://docs.fast.ai/tutorial.tabular.html
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/export.pkl'),Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/adult.csv')]

In [4]:
# change the name of the csv inside of the path
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [5]:
# update categorical names (one or the other), continuous names (any number or value), and change y_names to be the column you want to predict
# also change path and csv name
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

In [7]:
# visualize data before training
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Local-gov,10th,Married-civ-spouse,Craft-repair,Husband,White,False,27.000000,206124.999885,6.0,<50k
1,Private,Some-college,Never-married,Sales,Own-child,White,False,20.000000,259495.996968,10.0,<50k
2,Local-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,False,30.000000,235271.000919,13.0,>=50k
3,Private,HS-grad,Divorced,Sales,Not-in-family,White,False,52.000000,143806.998829,9.0,<50k
4,State-gov,Some-college,Separated,Tech-support,Unmarried,White,False,42.000000,211915.000270,10.0,<50k
5,Private,HS-grad,Never-married,Adm-clerical,Unmarried,Black,False,32.000000,116055.002931,9.0,<50k
6,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,False,52.999999,155962.999286,13.0,<50k
7,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Asian-Pac-Islander,False,46.000000,104632.002664,9.0,>=50k
8,Self-emp-not-inc,HS-grad,Married-civ-spouse,Handlers-cleaners,Husband,White,False,41.000000,170214.000372,9.0,<50k
9,Private,HS-grad,Divorced,Adm-clerical,Unmarried,White,False,30.000000,226943.000736,9.0,<50k


In [8]:
# create a model
learn = tabular_learner(dls, metrics=accuracy)

In [9]:
# fit for one "epoch" or entire loop of all of the data (change the number and see what happens!)
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.358423,0.354183,0.834459,00:04


In [10]:
# see predicted vs real results in new column
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,2.0,3.0,5.0,1.0,5.0,1.0,-0.484006,0.025256,-1.206859,0.0,0.0
1,5.0,10.0,3.0,5.0,1.0,5.0,1.0,-0.410468,-0.134635,1.145659,1.0,1.0
2,5.0,3.0,5.0,13.0,2.0,3.0,1.0,-0.778158,2.071850,-0.814773,0.0,0.0
3,5.0,16.0,3.0,4.0,1.0,5.0,1.0,-0.042778,-0.820669,-0.030600,1.0,0.0
4,5.0,2.0,5.0,4.0,5.0,5.0,1.0,-1.145848,1.546116,-1.206859,0.0,0.0
5,5.0,10.0,5.0,5.0,4.0,5.0,1.0,-1.072310,-0.922227,1.145659,0.0,0.0
6,5.0,16.0,3.0,5.0,1.0,5.0,1.0,-0.557544,0.106368,-0.030600,1.0,1.0
7,5.0,16.0,5.0,7.0,2.0,5.0,1.0,-1.292924,-1.413638,-0.030600,0.0,0.0
8,5.0,16.0,5.0,9.0,4.0,5.0,1.0,-1.366462,0.252734,-0.030600,0.0,0.0


In [11]:
# predict on a row in df (in this case, row 0)
row, clas, probs = learn.predict(df.iloc[0])
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,#na#,#na#,#na#,#na#,#na#,#na#,False,49.0,101320.001323,12.0,<50k


In [ ]:
# get predictions on a new_dataset using the trained model
# new_dataset should have same column names as the one you trained the model on
dl = learn.dls.test_dl(new_dataset)
learn.get_preds(dl=dl)